# 格式化输入与输出

`RunnableParallel`原语本质上是一个字典，它的值是可运行的。

`RunnableParallel`将它的整体输入作为参数来调用所有值，并进行计算，最终结果是一个字典。每个值的计算结果在其对应的`key`下。

In [1]:
import os

# 初始化
API_KEY = os.getenv("UNION_API_KEY")
BASE_URL = os.getenv("UNION_BASE_URL")

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

# 类型转换自动将字典变成一个`RunnableParallel`对象，然后并行执行`retriever`和`RunnablePassthrough`
# 结果是一个字典，包含`context`和`question`两个键，对应着`retriever`和`RunnablePassthrough`的输出
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

## 使用`itemgetter`快捷方式

与`RunnableParallel`结合时，可以使用Python的`itemgetter`从map中提取数据.

In [3]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

# 使用`itemgetter`分别提取`question`、`language`字段，并使用`retriever`进行检索。
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

## Parallelize Steps

`RunnableParallel`（又名 `RunnableMap`）可以轻松并行执行多个 `Runnable`，并将这些 `Runnable` 的输出作为map返回。

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why don't bears like fast food?\n\nBecause they can't catch it!", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3fefccc8-8daf-40f0-92b3-d1f3a7b85e5f-0'),
 'poem': AIMessage(content="In the forest's embrace, a bear roams free,\nStrength and grace entwined, wild majesty.", response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 15, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ee2060f1-4e2b-441e-8a8e-17e5fb04e2f2-0')}

# Parallelism

In [5]:
%%timeit

joke_chain.invoke({"topic": "bear"})

1.04 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

poem_chain.invoke({"topic": "bear"})

904 ms ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

map_chain.invoke({"topic": "bear"})

1.07 s ± 277 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
